In [2]:
import numpy as np
import scipy
import scipy.linalg
from scipy.special import gamma
import tensorflow as tf

import plotly.graph_objs as go
import plotly.express as px
from tqdm import trange
import matplotlib.pyplot as plt

In [3]:
def sphere_function(X):
    return np.sum(X**2, axis=1)

In [4]:
def ellipsoid_function(X, a=2):
    n = X.shape[-1]
    return np.sum((a**(np.arange(n)/(n-1)))*X**2, axis=1)

In [5]:
def rastrigin_function(X):
    return 10.0 * X.shape[1] + np.sum(X**2, axis=1) - 10.0 * np.sum(np.cos(2 * np.pi * X), axis=1)

In [6]:
def schwefel_function(X):
    return 418.9829 * X.shape[1] - np.sum(X * np.sin(np.sqrt(np.abs(X))), axis=1)

In [7]:
def griewank_function(X):
    return 1 + np.sum(X**2 / 4000, axis=1) - np.prod(np.cos(X / np.sqrt(np.linspace(1, X.shape[1], X.shape[1]))), axis=1)

In [8]:
def cigar_function(X, a = 3):
    if len(X.shape) == 1:
        X = X[np.newaxis,:]
    x1 = X[:,0]**2
    x2 = np.sum(a*X[:,1:]**2, axis = 1)
    return x1+x2

In [9]:
def discus_function(X, a = 3):
    if len(X.shape) == 1:
        X = X[np.newaxis,:]
    x1 = a*X[:,0]**2
    x2 = np.sum(X[:,1:]**2, axis = 1)
    return x1+x2

In [10]:
def cigar_discus_function(X, a = 2):
    if len(X.shape) == 1:
        X = X[np.newaxis,:]
    x1 = a*X[:,0]**2
    x2 = np.sum((a**0.5)*X[:,1:-1]**2, axis = 1)
    x3 = X[:,-1]**2
    return x1+x2+x3

In [11]:
def parab_ridge_function(X):
    if len(X.shape) == 1:
        X = X[np.newaxis,:]
    x1 = - X[:,0]
    x2 = 100*np.sum(X[:,1:]**2, axis = 1)
    return x1+x2

In [12]:
def two_axes_function(X, a = 5):
    if len(X.shape) == 1:
        X = X[np.newaxis,:]
    n = X.shape[-1]
    k = n//2
    x1 = np.sum(a*X[:,:k]**2, axis = 1)
    x2 = np.sum(X[:,k:]**2, axis = 1)
    return x1+x2

In [13]:
class ES():
    def __init__(self, domain, dim, population_size, offspring_size, fitness, max_iter, plus=False, parent_random=False):
        assert population_size <= offspring_size
        if type(domain) is tuple:
            self.domain = np.array([domain] * dim)
        else:
            self.domain = np.array(domain)

        self.dim = dim
        self.mu = population_size
        self.lam = offspring_size
        lr = 1 #K 
        self.tau0, self.tau = lr/np.sqrt(2 * np.sqrt(self.dim)), lr/np.sqrt(2*self.dim)
        self.fitness = fitness
        self.max_iter = max_iter
        self.plus = plus
        if parent_random:
            self.parent_selection = self.parent_selection_random
        else:
            self.parent_selection = self.parent_selection_roulette

    def initial_individual(self):
        x = np.random.uniform(self.domain[:,0], self.domain[:,1], size=self.dim)
        sigma = np.random.uniform(3, 8, size=self.dim)

        return np.vstack((x, sigma))

    def random_individual(self):
        x = np.random.uniform(self.domain[:,0], self.domain[:,1], size=self.dim)
        sigma = np.random.uniform(0.5, 1.5, size=self.dim)

        return np.vstack((x, sigma))

    def generate_population(self, initial=False):
        P = []
        for _ in range(self.mu):
            if initial:
                P.append(self.initial_individual())
            else:
                P.append(self.random_individual())
        P = np.array(P)
        return np.array(P)

    def mutate(self, P):
        xs, sigmas = P[:, 0], P[:, 1]
        assert type(sigmas) == np.ndarray
        mutated = []

        for i in range(len(P)):
            eps0 = np.random.normal(0, self.tau0**2)
            eps = np.random.normal(0, self.tau**2)
            multiplier = np.exp(eps0 + eps)
            
            new_sigma = sigmas[i] * multiplier

            new_x = xs[i] + np.random.normal(0, new_sigma**2, size=self.dim)
            
            its = 0
            while (np.any(new_x < self.domain[:,0]) or np.any(new_x > self.domain[:,1])) and its < 4:
                its += 1
                new_x = xs[i] + np.random.normal(0, new_sigma**2, size=self.dim)
            
            if (np.any(new_x < self.domain[:,0]) or np.any(new_x > self.domain[:,1])):
                new_x, new_sigma = self.random_individual()
            
            mutated.append(np.vstack((new_x, new_sigma)))
            
        return np.array(mutated)

    def replacement(self, P, P_costs, pc, pc_costs):
        if(self.plus):
            combined = np.concatenate((P, pc), axis=0)
            assert combined.shape == (self.mu + self.lam, 2, self.dim), f"{combined.shape}"
            combined_scores = np.concatenate((P_costs, pc_costs))
            indices = np.argsort(combined_scores)
            assert combined_scores[indices[0]] <= combined_scores[indices[-1]]
            P = combined[indices[:self.mu]]
        else:
            scores = np.array([self.fitness(p[0]) for p in pc])
            indices = np.argsort(scores)
            P = pc[indices[:self.mu]]
        return P

    def parent_selection_random(self, P, score):
        return P[np.random.choice(len(P), self.lam, replace=True)]

    def parent_selection_roulette(self, P, score):
        if np.min(score) == np.max(score):
            idxs = np.random.choice(len(P), size=self.lam, replace=True)
        else:
            std_costs = (score - np.min(score)) / (score - np.min(score)).max()
            p_costs = (1 - std_costs)
            idxs = np.random.choice(len(P), p=p_costs / sum(p_costs), size=self.lam, replace=True)
        return P[idxs].copy()

    def run(self):
        P = self.generate_population(initial=True)
        assert P.shape == (self.mu, 2, self.dim) 
        score = np.array([self.fitness(p[0]) for p in P])
        best_score = np.min(score)
        best_sigma = []
        best_individual = P[np.argmin(score)]

        iter_min, iter_mean, iter_max = [], [], []
        range_ = trange(int(self.max_iter), position=0, leave=True)
        try:
            for _iter in range_:
                parents = self.parent_selection(P, score)
                assert parents.shape == (self.lam, 2, self.dim)

                mutation = self.mutate(parents)
                assert mutation.shape == (self.lam, 2, self.dim)

                P = self.replacement(P, score, mutation, [self.fitness(p[0]) for p in mutation])
                assert P.shape == (self.mu, 2, self.dim), f'{P.shape}'

                score = np.array([self.fitness(p[0]) for p in P])

                min_score_index = np.argmin(score)
                min_score = np.min(score)
                if min_score < best_score:
                    best_score = min_score
                    best_individual = P[min_score_index]
                
                # print(f'Iter: {_iter}, best: {best_score}')
                iter_min.append(np.min(score))
                iter_mean.append(np.mean(score))
                iter_max.append(np.max(score))
                #calculate mean of P[1]:
                best_sigma.append(np.mean(P[:, 1]))
        except KeyboardInterrupt:
            pass

        return best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max

def showcase(best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max, problem_name, global_minima):
    print(f'Best individual: {best_individual[0]}')
    print(f'Best score: {best_score}')
    plt.plot(iter_mean, label='mean')
    plt.plot(iter_min, label='min')
    global_minima_array = np.full(len(iter_min), global_minima)
    plt.plot(global_minima_array, label='min_global', color='r')
    plt.legend()
    plt.title(problem_name)
    plt.show()

    plt.plot(best_sigma, label='sigma')
    plt.title('Sigma')
    plt.show()

def ESPlusRandom(domain, dim, population_size, offspring_size, fitness, max_iter, problem_name, global_minima):
    model = ES(domain, dim, population_size, offspring_size, fitness, max_iter, plus=True, parent_random=True)
    best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max = model.run()
    showcase(best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max, problem_name, global_minima)

def ESPlusRoulette(domain, dim, population_size, offspring_size, fitness, max_iter, problem_name, global_minima):
    model = ES(domain, dim, population_size, offspring_size, fitness, max_iter, plus=True, parent_random=False)
    best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max = model.run()
    showcase(best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max, problem_name, global_minima)

def ESCommaRandom(domain, dim, population_size, offspring_size, fitness, max_iter, problem_name, global_minima):
    model = ES(domain, dim, population_size, offspring_size, fitness, max_iter, plus=False, parent_random=True)
    best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max = model.run()
    showcase(best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max, problem_name, global_minima)

def ESCommaRoulette(domain, dim, population_size, offspring_size, fitness, max_iter, problem_name, global_minima):
    model = ES(domain, dim, population_size, offspring_size, fitness, max_iter, plus=False, parent_random=False)
    best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max = model.run()
    showcase(best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max, problem_name, global_minima)

In [ ]:
class CMA_ES:
    def __init__(self, x0, sigma, maxfevals = 10000, popsize = None, weights = None, domain = None):
        N = x0.shape[0]
        self.dimension = N
        self.chiN = N**0.5 * (1 - 1. / (4 * N) + 1. / (21 * N**2))
        self.lam = 4 + int(3 * np.log(N)) if not popsize else popsize
        print(f"Popsize: {self.lam}")
        self.mu = int(self.lam / 2)
        self.shape = tf.cast((self.lam, N), tf.int32)
        
        if weights:
            self.weights = weights
        else:
            self.weights = np.array([np.log(self.lam / 2 + 0.5) - np.log(i + 1) if i < self.mu else 0
                        for i in range(self.lam)])
            self.weights /= np.sum(self.weights)
        if(self.weights.shape == (self.lam,)):
            self.weights = self.weights[:, np.newaxis]
        if domain:
            self.domain = domain
        self.mueff = np.sum(self.weights)**2 / np.sum(self.weights**2)
        
        self.cc = (4 + self.mueff/N) / (N+4 + 2 * self.mueff/N)
        self.cs = (self.mueff + 2) / (N + self.mueff + 5)
        self.c1 = 2 / ((N + 1.3)**2 + self.mueff) 
        # self.cmu = min([1 - self.c1, 2 * (self.mueff - 2 + 1/self.mueff) / ((N + 2)**2 + self.mueff)])
        self.cmu = 2 * (self.mueff - 2 + 1 / self.mueff) / ((N + 2)**2 + 2 * self.mueff / 2)
        # self.damps = 2 * self.mueff/self.lam + 0.3 + self.cs
        self.damps = 1 + 2 * max(0, np.sqrt((self.mueff - 1)/(N + 1)) - 1) + self.cs

        self.xmean = np.array(x0[:])
        self.sigma = sigma
        self.pc = np.zeros(N) 
        self.ps =np.zeros(N) 
        self.lazy_gap_evals = 0.5 * N * self.lam * (self.c1 + self.cmu)**-1 / N**2
        self.maxfevals = maxfevals
        self.C = np.identity(N)
        self.counteval = 0 
        self.fitvals = []   
        self.best = (x0, None)
        self.condition_number = 1
        self.eigen_values = np.ones(N)
        self.eigen_vectors = np.identity(N)
        self.updated_eval = 0
        self.inv_sqrt = np.ones(N)
        self.B = np.eye(self.dimension)
        self.D = np.eye(self.dimension)

    def _update_eigensystem(self, current_eval, lazy_gap_evals):
        if current_eval <= self.updated_eval + lazy_gap_evals:
            return self
        self.eigen_values, self.eigen_vectors = np.linalg.eig(self.C)
        self.inv_sqrt = self.eigen_vectors @ np.diag(self.eigen_values**-0.5) @ self.eigen_vectors.T
        self.condition_number = self.eigen_values.max() / self.eigen_values.min()
         
    def sample(self):
        z = tf.random.normal(self.shape, dtype=tf.float64)
        z = np.array(z)
        y = z @ (self.B @ self.D)
        x = self.xmean + self.sigma * y
        return x
    
    def update(self, x, fitvals):
        """Zaktualizuj wartoĹci uzyskanych parametrĂłw"""
        self.counteval += fitvals.shape[0] 
        #check if x in domain, if its not add 1e8 to fitval of this x
        if self.domain:
            for i, xi in enumerate(x):
                if np.any(xi < self.domain[0]) or np.any(xi > self.domain[1]):
                    fitvals[i] += 1e8

        
        #------------------------------------------------------------------------------------------------------
        idx = np.argsort(fitvals)
        x_sorted = x[idx]
        self.fitvals = fitvals[idx] 
        self.best = (x[0], self.fitvals[0])

        xdiff = x_sorted - self.xmean
        x_mean = np.sum(xdiff * self.weights, axis=0)
        m = self.xmean + x_mean
        #------------------------------------------------------------------------------------------------------
        y_mean = x_mean / self.sigma 
        pc = (1 - self.cc) * self.pc + np.sqrt(self.cc * (2 - self.cc) * self.mueff) * y_mean
        pcmatrix = pc[:, np.newaxis]

        C_m = np.array([e[:, np.newaxis] * e.T for e in (xdiff / self.sigma)])
        y_s = np.sum(C_m * self.weights[:, np.newaxis], axis=0)

        C = (1 - self.c1 - self.cmu) * self.C + self.c1 * pcmatrix * pcmatrix.T + self.cmu * y_s

        C = (C + C.T)/2.0
        
        #--------------------------------------------------------------------------------------------------------

        D_inv = np.diag(np.reciprocal(np.diag(self.D)))
        C_inv_squared = (self.B @ D_inv) @ (self.B.T)
        C_inv_squared_y = np.squeeze(C_inv_squared @ y_mean[:, np.newaxis])  
        ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * C_inv_squared_y  

        sigma = self.sigma * np.exp((self.cs / self.damps) * ((np.linalg.norm(ps) / self.chiN) - 1))

        #--------------------------------------------------------------------------------------------------------
        u, B, _ = tf.linalg.svd(C)
        u = np.array(u)
        B = np.array(B)
        diag_D = np.sqrt(u)
        D = np.diag(diag_D)

        #--------------------------------------------------------------------------------------------------------

        self.pc = pc
        self.ps = ps
        self.C = C
        self.sigma = sigma
        self.B = B
        self.D = D
        self.xmean = m
        
    def terminate(self):
        """ZakoĹcz algorytm"""
        if self.counteval <= 0:
            return False
        if self.counteval >= self.maxfevals:
            return True
        if self.condition_number > 1e13:
            return True
        if self.sigma * np.max(self.eigen_values)**0.5 < 1e-13:
            return True
        return False

In [52]:
def optimize(func, x0, sigma, maxfevals = 1000, popsize = None, weights = None, domain = None):
    cma_es = CMA_ES(x0, sigma, maxfevals, popsize, weights, domain)
    res = []
    cntr = 0
    while not cma_es.terminate():
        cntr+=1
        x = cma_es.sample()
        f_eval = func(x)
        cma_es.update(x, f_eval)
        res.append(cma_es.best)
        if cntr % 100 == 0:
            print(f"Iteration {cntr:5d}: {res[-1][1]}")
    return res

def optimize_and_plot(f, sigma = 1, d = 10, popsize = None, maxfevals = 1000, domain = None):
    x0 = np.repeat(100.0, d)
    res = optimize(f, x0, sigma, popsize = popsize, maxfevals = maxfevals, domain = domain)
    print(f"Best: {res[-1][0]}, value: {res[-1][1]}")
    y = np.array([nd for st, nd in res])
    fig = px.line(x = np.arange(y.shape[0]) + 1, y = y)
    fig.show()

zadanie a

sphere

In [ ]:
optimize_and_plot(sphere_function, d = 5, domain = (-5.12, 5.12))

Popsize: 8
Iteration   100: 0.11296835088306005
Best: [ 0.00897296 -0.01118874  0.0141083   0.01115365  0.01456242], value: 0.00017345987156185596


In [61]:
optimize_and_plot(sphere_function, d = 10, domain = (-5.12, 5.12), popsize=500, maxfevals=100000, sigma=50)

Popsize: 500
Iteration   100: 2.565484178082345e-24
Iteration   200: 1.1116905548243855e-53
Best: [ 2.19209011e-28  1.84947150e-27  8.22180070e-28 -8.85030556e-28
  4.58083187e-27  7.92380322e-29  2.51572127e-27  4.39625016e-27
  7.23832404e-28 -2.11283271e-27], value: 1.1116905548243855e-53


In [62]:
optimize_and_plot(sphere_function, d = 20, domain = (-5.12, 5.12), popsize=500, maxfevals=100000, sigma=50)

Popsize: 500
Iteration   100: 1.0074414974675979e-05
Iteration   200: 9.4711912697933e-16
Best: [-2.07764635e-09 -1.48326008e-09  2.32513750e-08  1.07522970e-08
 -7.35483616e-09  1.69488894e-09 -3.58773393e-09  1.33553985e-08
 -7.27107448e-09 -6.86556064e-09  1.21479613e-08 -2.26291528e-09
  4.37240450e-09  1.50549080e-09  1.29587863e-08  1.66815180e-09
  2.55015707e-09 -2.54513715e-09  1.62296154e-08 -7.12376842e-09], value: 9.4711912697933e-16


In [65]:
optimize_and_plot(sphere_function, d = 50, domain = (-5.12, 5.12), popsize=500, maxfevals=100000, sigma=10)

Popsize: 500
Iteration   100: 65.16483938145774
Iteration   200: 0.012014002947622603
Best: [-0.0123654  -0.02751959 -0.02535668  0.01133357 -0.01443183  0.00437269
 -0.00716125 -0.0199278  -0.01624611 -0.00690688 -0.0092556   0.00742177
  0.02258037  0.00492348 -0.0026103  -0.00449763  0.02645549  0.03717039
 -0.00812592 -0.00059073 -0.01232751 -0.00934995  0.00355404 -0.0081335
  0.03481337  0.00184162  0.00912882 -0.02752738 -0.01733219  0.01780938
  0.00763403 -0.00533364 -0.00310083  0.00519071  0.0459172  -0.00833665
  0.01832819  0.01224867  0.01856971  0.00503402  0.01147402  0.0102055
 -0.00197663 -0.00946581 -0.0008635  -0.02255813 -0.0149894  -0.03634151
 -0.01143789  0.00864863], value: 0.012014002947622603


In [66]:
optimize_and_plot(sphere_function, d = 50, domain = (-5.12, 5.12), popsize=500, maxfevals=100000, sigma=50)

Popsize: 500
Iteration   100: 223.1662170017617
Iteration   200: 0.01914381707098044
Best: [-0.01088467 -0.00953298  0.02812747  0.01953265 -0.02831844 -0.05306989
 -0.03924954  0.01762111 -0.05317746  0.0642242  -0.04054368 -0.01192632
 -0.00376269 -0.04103799 -0.05456586  0.02498847  0.04273514 -0.02910183
  0.01478626 -0.02675488 -0.01798077 -0.0558744  -0.00613476 -0.01800981
 -0.01765979  0.01533378 -0.03431414  0.03783389  0.01801991  0.05684933
 -0.00424863  0.04437879 -0.00965041 -0.02434196  0.04618046  0.02576718
  0.04857366 -0.01699527  0.00420123 -0.01313729 -0.01495064 -0.00062256
 -0.03557129  0.01136145  0.046752    0.03467868 -0.01238935 -0.01308043
 -0.0164344   0.01312719], value: 0.01914381707098044


In [67]:
optimize_and_plot(sphere_function, d = 50, domain = (-5.12, 5.12), popsize=500, maxfevals=100000, sigma=100)

Popsize: 500
Iteration   100: 100000515.17592068
Iteration   200: 0.0750121219985171
Best: [-0.00802186  0.0343857   0.0185348  -0.09489016  0.04993549  0.01038053
  0.00151837 -0.0703109  -0.0520414  -0.11364129 -0.06989439  0.02683528
  0.02923938  0.18086208 -0.01445183 -0.02334671 -0.10950222 -0.02476247
  0.10444692  0.02401662  0.0063338  -0.05159644  0.06077676  0.08832254
 -0.01832252  0.06856187 -0.00895716  0.04994334 -0.05761813  0.04526531
  0.06887865 -0.02521067  0.04888087 -0.00481565 -0.07852663  0.03622903
  0.03001375 -0.03106919  0.00210517  0.0258669   0.08609623 -0.06201399
  0.07022     0.08144091  0.06467075 -0.03828325  0.07442462  0.0063931
 -0.02839086 -0.03000146], value: 0.0750121219985171


rastrigin

In [72]:
optimize_and_plot(rastrigin_function, d = 5, domain = (-5.12, 5.12), popsize=100, maxfevals=100000, sigma=10)

Popsize: 100
Iteration   100: 1.4210854715202004e-14
Iteration   200: 0.0
Iteration   300: 0.0
Iteration   400: 0.0
Iteration   500: 0.0
Iteration   600: 0.0
Iteration   700: 0.0
Iteration   800: 0.0
Iteration   900: 0.0
Iteration  1000: 0.0
Best: [ 8.52406372e-10  2.73164181e-09 -2.58285545e-09  4.49578509e-10
  3.67961448e-10], value: 0.0


In [74]:
optimize_and_plot(rastrigin_function, d = 10, domain = (-5.12, 5.12), popsize=100, maxfevals=100000, sigma=10)

Popsize: 100
Iteration   100: 0.4132897965378248
Iteration   200: 2.8421709430404007e-13
Iteration   300: 0.0
Iteration   400: 0.0
Iteration   500: 0.0
Iteration   600: 0.0
Iteration   700: 0.0
Iteration   800: 0.0
Iteration   900: 0.0
Iteration  1000: 0.0
Best: [ 2.24881573e-09  5.01785136e-10  3.41380979e-10  1.49952023e-09
  1.10795595e-09 -6.57411093e-10  1.41531721e-09  2.08803318e-10
 -4.91845731e-10  2.26500333e-09], value: 0.0


In [82]:
optimize_and_plot(rastrigin_function, d = 20, domain = (-5.12, 5.12), popsize=100, maxfevals=100000, sigma=50)

Popsize: 100
Iteration   100: 91.24909681015887
Iteration   200: 1.0098320617737784
Iteration   300: 0.9949590571884244
Iteration   400: 0.9949590570932685
Iteration   500: 0.9949590570932401
Iteration   600: 0.9949590570932685
Iteration   700: 0.9949590570932401
Iteration   800: 0.9949590570932685
Iteration   900: 0.9949590570932685
Iteration  1000: 0.9949590570932685
Best: [-1.22032905e-09 -2.54585696e-09 -3.17719523e-10  1.96644486e-10
 -4.63651491e-10  4.55046421e-10 -1.03413162e-10  7.82927333e-10
 -1.58914199e-09  1.01039068e-09 -5.75610391e-10 -1.15351030e-10
  9.94958639e-01  1.10933195e-09  8.42638031e-10  2.14211633e-09
 -1.22664853e-09 -3.18109966e-09 -4.53959842e-09 -8.06569666e-10], value: 0.9949590570932685


In [89]:
optimize_and_plot(rastrigin_function, d = 50, domain = (-5.12, 5.12), popsize=50, maxfevals=150000, sigma=50)

Popsize: 50
Iteration   100: 534.9802487653501
Iteration   200: 416.5271298942862
Iteration   300: 374.6097994617602
Iteration   400: 32.26634515564422
Iteration   500: 31.838744448334126
Iteration   600: 31.838679756872523
Iteration   700: 31.83867975134757
Iteration   800: 31.838679751347115
Iteration   900: 31.83867975134706
Iteration  1000: 31.838679751347115
Iteration  1100: 31.838679751347115
Iteration  1200: 31.838679751347115
Iteration  1300: 31.83867975134706
Iteration  1400: 31.838679751347115
Iteration  1500: 31.83867975134706
Iteration  1600: 31.838679751347115
Iteration  1700: 31.838679751347115
Iteration  1800: 31.838679751347115
Iteration  1900: 31.838679751347115
Iteration  2000: 31.838679751347115
Iteration  2100: 31.838679751347115
Iteration  2200: 31.838679751347115
Iteration  2300: 31.838679751347115
Iteration  2400: 31.838679751347172
Iteration  2500: 31.838679751347115
Iteration  2600: 31.83867975134706
Iteration  2700: 31.83867975134706
Iteration  2800: 31.838679

In [90]:
optimize_and_plot(rastrigin_function, d = 50, domain = (-5.12, 5.12), popsize=50, maxfevals=150000, sigma=200)

Popsize: 50
Iteration   100: 761.927283054321
Iteration   200: 439.89773017016336
Iteration   300: 240.66119673488697
Iteration   400: 37.8494143586413
Iteration   500: 37.80842792204686
Iteration   600: 37.808424018574726
Iteration   700: 37.808424018268795
Iteration   800: 37.80842401826874
Iteration   900: 37.80842401826868
Iteration  1000: 37.80842401826868
Iteration  1100: 37.80842401826874
Iteration  1200: 37.80842401826868
Iteration  1300: 37.80842401826868
Iteration  1400: 37.808424018268624
Iteration  1500: 37.80842401826868
Iteration  1600: 37.80842401826868
Iteration  1700: 37.80842401826874
Iteration  1800: 37.80842401826868
Iteration  1900: 37.80842401826868
Iteration  2000: 37.80842401826874
Iteration  2100: 37.80842401826868
Iteration  2200: 37.80842401826868
Iteration  2300: 37.808424018268624
Iteration  2400: 37.80842401826868
Iteration  2500: 37.80842401826868
Iteration  2600: 37.80842401826868
Iteration  2700: 37.80842401826868
Iteration  2800: 37.80842401826868
Iter

In [88]:
optimize_and_plot(rastrigin_function, d = 50, domain = (-5.12, 5.12), popsize=5000, maxfevals=1500000, sigma=100)

Popsize: 5000
Iteration   100: 326.8812234708795
Iteration   200: 0.7033221971543071
Iteration   300: 1.904084001580486e-09
Best: [-2.47520732e-07  7.71597095e-07 -2.21373922e-07  1.80128218e-06
 -3.06765780e-07  5.06139149e-07  3.14521262e-08  3.26830657e-07
  1.09335953e-06 -6.52958745e-07 -1.81996452e-07  6.25397175e-07
 -8.12880148e-07  1.66147221e-06  1.53124170e-06  5.12597283e-07
 -1.55213403e-07  3.32491446e-07 -1.56311857e-06  1.12081104e-06
 -5.95813191e-07 -6.90476224e-07 -2.42639186e-07 -8.84894990e-08
 -2.84979813e-07 -8.06969607e-07 -5.37707997e-08 -1.25662438e-06
  2.90609144e-07 -5.62943341e-07  1.20643637e-06  1.63521511e-06
  1.16404927e-06  2.99232526e-07 -3.31779376e-07 -2.05213653e-07
 -2.33958732e-07 -1.38110724e-07  8.06169922e-07 -1.86974443e-07
  4.93960086e-07  2.25848145e-07 -9.02982758e-09  6.87435294e-07
  5.16238597e-07 -2.63542699e-07 -9.97279397e-07  3.90147215e-07
  2.16125443e-07  8.60724391e-08], value: 1.904084001580486e-09


In [91]:
optimize_and_plot(rastrigin_function, d = 50, domain = (-5.12, 5.12), popsize=5000, maxfevals=1500000, sigma=50)

Popsize: 5000
Iteration   100: 322.0100034325797
Iteration   200: 0.5741080366418032
Iteration   300: 1.9966819309047423e-09
Best: [-1.04509168e-06  8.60208479e-07  4.60254159e-07  1.72012054e-07
  4.47640190e-07  3.50251234e-08  5.14629633e-08  6.83042230e-07
 -4.08905880e-07  4.04449333e-07 -4.39214198e-08 -4.79132291e-07
  1.40678657e-06  9.15452448e-07 -1.10391356e-06 -1.54933115e-07
  6.28262048e-09 -2.17335776e-07  5.14697396e-08  6.08938739e-07
  9.46531329e-08  4.80673660e-07  1.23796752e-08 -6.33144626e-07
 -1.45589917e-06  4.53283772e-08  8.77256908e-07  2.06148966e-07
 -6.28490733e-07 -3.80171698e-07  7.86219844e-07 -1.71201559e-07
 -3.13273918e-07 -2.51311160e-07 -5.27129978e-07  8.42395612e-07
  4.04448122e-07 -4.58890185e-07  4.92145845e-07 -1.58382767e-07
 -1.03824582e-08 -5.15221064e-07 -4.38874725e-07  4.28410088e-07
  1.84531332e-07 -1.52027624e-07 -5.54744049e-08 -5.10085303e-07
  1.81623941e-07 -1.60428637e-07], value: 1.9966819309047423e-09


Griewank    

In [99]:
optimize_and_plot(griewank_function, d = 5, domain = (-600, 600), popsize=100, maxfevals=150000, sigma=10)

Popsize: 100
Iteration   100: 0.00739604033411545
Iteration   200: 0.007396040334114895
Iteration   300: 0.007396040334114895
Iteration   400: 0.007396040334114895
Iteration   500: 0.007396040334114895
Iteration   600: 0.007396040334114895
Iteration   700: 0.007396040334114895
Iteration   800: 0.007396040334114895
Iteration   900: 0.007396040334114895
Iteration  1000: 0.007396040334114895
Iteration  1100: 0.007396040334114895
Iteration  1200: 0.007396040334114895
Iteration  1300: 0.007396040334114895
Iteration  1400: 0.007396040334114895
Iteration  1500: 0.007396040334114895
Best: [ 3.14002264e+00 -4.43844448e+00 -1.70983278e-09  4.41166042e-09
 -1.96148694e-09], value: 0.007396040334114895


In [100]:
optimize_and_plot(griewank_function, d = 10, domain = (-600, 600), popsize=100, maxfevals=150000, sigma=10)

Popsize: 100
Iteration   100: 1.1920903478124245e-05
Iteration   200: 0.0
Iteration   300: 0.0
Iteration   400: 0.0
Iteration   500: 0.0
Iteration   600: 0.0
Iteration   700: 0.0
Iteration   800: 0.0
Iteration   900: 0.0
Iteration  1000: 0.0
Iteration  1100: 0.0
Iteration  1200: 0.0
Iteration  1300: 0.0
Iteration  1400: 0.0
Iteration  1500: 0.0
Best: [-6.07678172e-09  8.86652054e-09 -7.12785824e-09  7.60756655e-09
 -5.44058009e-09 -4.22363030e-09 -1.93558506e-09 -1.25502452e-08
  5.31899909e-09  6.19503311e-09], value: 0.0


In [101]:
optimize_and_plot(griewank_function, d = 20, domain = (-600, 600), popsize=100, maxfevals=150000, sigma=10)

Popsize: 100
Iteration   100: 0.00399536462624106
Iteration   200: 4.019131358834471e-07
Iteration   300: 7.955514025326238e-11
Iteration   400: 1.432187701766452e-14
Iteration   500: 0.0
Iteration   600: 0.0
Iteration   700: 0.0
Iteration   800: 0.0
Iteration   900: 0.0
Iteration  1000: 0.0
Iteration  1100: 0.0
Iteration  1200: 0.0
Iteration  1300: 0.0
Iteration  1400: 0.0
Iteration  1500: 0.0
Best: [ 2.63746342e-09 -5.05933093e-09  5.93846166e-09  3.35422813e-09
  3.73957597e-09 -3.08177189e-09  2.39013963e-09 -1.31663906e-08
  3.19713897e-09  2.20325577e-08  7.42037525e-09 -8.51902564e-09
 -1.72618703e-08  3.77718395e-09 -6.97323886e-09 -3.68738902e-08
  9.22136415e-09  3.09232325e-08 -3.80165924e-09  3.26751226e-08], value: 0.0


In [102]:
optimize_and_plot(griewank_function, d = 50, domain = (-600, 600), popsize=100, maxfevals=150000, sigma=10)

Popsize: 100
Iteration   100: 1.0162097810892512
Iteration   200: 0.010492613820653363
Iteration   300: 6.003990039571416e-05
Iteration   400: 8.355762898260011e-07
Iteration   500: 4.575354506375362e-08
Iteration   600: 2.7395374857519528e-09
Iteration   700: 2.0136592393527053e-10
Iteration   800: 1.1994627513445266e-11
Iteration   900: 5.823119764158946e-13
Iteration  1000: 1.63202784619898e-14
Iteration  1100: 5.551115123125783e-16
Iteration  1200: 1.1102230246251565e-16
Iteration  1300: 1.1102230246251565e-16
Iteration  1400: 1.1102230246251565e-16
Iteration  1500: 1.1102230246251565e-16
Best: [ 3.16834612e-09 -5.18855821e-09  1.03327858e-08 -1.53715296e-09
  5.64823502e-10  2.38585841e-08 -3.60223459e-09 -6.42501060e-09
  1.35398884e-08  3.85888143e-09  2.08579346e-08 -1.85300426e-08
 -1.70991697e-08  2.33079747e-08  1.35898132e-08 -2.22691064e-08
 -2.64883777e-08 -1.39370971e-08  4.21172215e-09  3.75350738e-09
 -2.84100807e-08 -2.06973066e-08  1.77721039e-09  2.96969938e-08
 -2.

In [104]:
optimize_and_plot(griewank_function, d = 50, domain = (-600, 600), popsize=5000, maxfevals=1500000, sigma=50)

Popsize: 5000
Iteration   100: 0.006919538695006278
Iteration   200: 3.0619563647915626e-06
Iteration   300: 3.3479112038214964e-08
Best: [-1.66406665e-04  5.37089441e-05 -5.54734589e-05  1.83590385e-05
 -2.05861451e-04  1.17287772e-04 -1.05102164e-04 -2.16239549e-04
  9.43383699e-05 -2.83626222e-04 -2.30630465e-04 -2.62145611e-05
 -1.20159412e-04 -1.19259452e-04 -1.08310016e-04 -2.39052870e-04
 -2.44357943e-04  6.15959748e-05 -1.40147204e-04  1.40381026e-04
  1.30088257e-04  2.05404091e-04 -4.84517783e-05  1.95790541e-04
  2.77112504e-04 -1.17983273e-04  1.14936856e-04 -1.74264917e-04
 -5.93968187e-05 -2.16282678e-04  4.64291463e-05  5.70486171e-05
  6.98407214e-05 -6.20851004e-05 -3.70266015e-04  1.01137344e-04
  1.89903843e-04  7.08033773e-05  3.60832377e-05  2.48922961e-04
  1.00351065e-04  1.37021110e-04  6.49048695e-05 -5.85438590e-05
 -8.26236589e-05  8.33597139e-05 -1.20450503e-04 -7.90208412e-04
 -2.58485683e-04  9.36879056e-04], value: 3.3479112038214964e-08


Schwefel

In [113]:
optimize_and_plot(schwefel_function, d = 5, domain = (-500, 500), popsize=500, maxfevals=150000, sigma=50)

Popsize: 500
Iteration   100: 434.27940241980923
Iteration   200: 434.27940241897704
Iteration   300: 434.27940241897704
Best: [420.96874635 420.96874626 420.96874632 203.81425259 203.81425178], value: 434.27940241897704


In [117]:
optimize_and_plot(schwefel_function, d = 10, domain = (-500, 500), popsize=100, maxfevals=150000, sigma=50)

Popsize: 100
Iteration   100: 2309.605057326097
Iteration   200: 2309.605057111698
Iteration   300: 2309.605057111698
Iteration   400: 2309.605057111698
Iteration   500: 2309.605057111698
Iteration   600: 2309.605057111698
Iteration   700: 2309.605057111698
Iteration   800: 2309.605057111698
Iteration   900: 2309.605057111698
Iteration  1000: 2309.605057111698
Iteration  1100: 2309.605057111698
Iteration  1200: 2309.605057111698
Iteration  1300: 2309.605057111698
Iteration  1400: 2309.605057111698
Iteration  1500: 2309.605057111698
Best: [203.81425245 203.81425264 203.81425259 203.81425261 203.81425251
 203.81425274 203.81425262 203.81425261  65.54786493 203.81425344], value: 2309.605057111698


In [118]:
optimize_and_plot(schwefel_function, d = 20, domain = (-500, 500), popsize=100, maxfevals=150000, sigma=50)

Popsize: 100
Iteration   100: 4827.21145379488
Iteration   200: 4639.099075862387
Iteration   300: 4639.0990673590495
Iteration   400: 4639.0990673590495
Iteration   500: 4639.0990673590495
Iteration   600: 4639.0990673590495
Iteration   700: 4639.0990673590495
Iteration   800: 4639.0990673590495
Iteration   900: 4639.0990673590495
Iteration  1000: 4639.0990673590495
Iteration  1100: 4639.0990673590495
Iteration  1200: 4639.0990673590495
Iteration  1300: 4639.0990673590495
Iteration  1400: 4639.0990673590495
Iteration  1500: 4639.0990673590495
Best: [ 203.81425376  420.96874533 -124.82935545  420.96874666  203.81425366
 -302.52493598  203.81425241  203.81425238  -25.87741723   65.54786506
 -124.8293561    65.54786477  203.81425237  203.81425322   65.54786515
  203.81425369 -124.82935682  203.81425357  203.81425376  203.8142532 ], value: 4639.0990673590495


In [121]:
optimize_and_plot(schwefel_function, d = 50, domain = (-500, 500), popsize=200, maxfevals=150000, sigma=50)

Popsize: 200
Iteration   100: 18613.130622036857
Iteration   200: 16913.078924879006
Iteration   300: 10657.925066189537
Iteration   400: 9968.957876205362
Iteration   500: 9968.881411834294
Iteration   600: 9968.88141157162
Iteration   700: 9968.881411571616
Best: [ 203.81425242  203.81425292   65.54786479 -124.82935586  203.81425255
  203.81425249 -302.52493535  203.81425283  203.81425296  203.81425238
 -124.82935604  203.81425247  203.8142527   203.81425287  203.81425264
 -124.8293565   420.96874619  203.81425264  203.81425251   65.54786511
  420.96874608  203.81425239  203.81425261  -25.87741725  203.81425218
  420.96874573  203.81425255  203.81425249  203.81425206  203.8142528
  203.81425237  203.81425289  203.81425274  420.96874646  203.81425232
  203.81425273 -124.82935644  203.81425258  203.81425335  203.8142524
  203.81425346 -302.52493588 -124.82935637  420.96874663  203.81425266
 -302.52493559 -124.82935645  420.96874644  420.96874689  203.81425217], value: 9968.881411571616

In [126]:
def compare(domain, func_CMA, func_ES, d, popsize_CMU = None, maxfevals = 1000, maxiter = 1000, popsize_ES = 1000, sigma_CMA = 50):
    x0 = np.repeat(100.0, d)
    res = optimize(func_CMA, x0, sigma=sigma_CMA, popsize = popsize_CMU, maxfevals = maxfevals, domain=domain)
    print(f"Best for CMU-ES: {res[-1][0]}, value: {res[-1][1]}")
    model = ES(domain, d, popsize_ES, popsize_ES*7, func_ES, maxiter, plus=True, parent_random=True)
    best_individual, best_sigma, best_score, iter_min, iter_mean, iter_max = model.run()
    print(f"Best for ES: {best_individual[0]}, value: {best_score}")
    

In [37]:
def Rastrigin(individual):
    d = len(individual)
    return 10*d + np.sum(individual**2 - 10*np.cos(2*np.pi*individual))

In [38]:
compare((-5.12, 5.12), rastrigin_function, Rastrigin, 5)

Popsize: 8
Iteration   100: 26.526135768298595


Best for CMU-ES: [-1.91380164 -0.14404647  2.8439568  -1.05143139 -0.73351972], value: 34.65463356516958


100%|██████████| 1000/1000 [02:38<00:00,  6.31it/s]

Best for ES: [ 1.54216257e-09 -9.94958637e-01 -1.78789387e-11 -9.94958639e-01
  2.07528308e-09], value: 1.9899181141865796


In [40]:
print(optimize(rastrigin_function, np.repeat(100.0, 50), 1, maxfevals = 1000000, popsize=500)[-1][1])

Popsize: 500
Iteration   100: 581.4029799149093
Iteration   200: 395.0238604022054
Iteration   300: 383.59923843410377
Iteration   400: 153.37074026559264
Iteration   500: 0.07563021113008972
Iteration   600: 1.180272982992392e-05
Iteration   700: 7.862581696826965e-10
Iteration   800: 5.684341886080802e-14
Iteration   900: 0.0
Iteration  1000: 0.0
Iteration  1100: 0.0
Iteration  1200: 0.0
Iteration  1300: 0.0
Iteration  1400: 0.0
Iteration  1500: 0.0
Iteration  1600: 0.0
Iteration  1700: 0.0
Iteration  1800: 0.0
Iteration  1900: 0.0
Iteration  2000: 0.0
0.0


In [122]:
def Schwefel(individual):
    d = len(individual)
    value = 418.9829*d
    for i in range(d):
        value -= individual[i] * np.sin(np.sqrt(np.abs(individual[i])))
    return value 

In [127]:
compare((-500, 500), schwefel_function, Schwefel, 50, popsize_CMU=500, maxfevals=1000000, maxiter=1000, popsize_ES=1000, sigma_CMA=50)

Popsize: 500
Iteration   100: 16715.178425472437
Iteration   200: 10811.505430837386
Iteration   300: 10778.058539624859
Iteration   400: 10778.052673092383
Iteration   500: 10778.05267244676
Iteration   600: 10778.05267244669
Iteration   700: 10778.05267244669
Iteration   800: 10778.05267244669
Iteration   900: 10778.05267244669
Iteration  1000: 10778.05267244669
Iteration  1100: 10778.05267244669
Iteration  1200: 10778.05267244669
Iteration  1300: 10778.05267244669
Iteration  1400: 10778.05267244669
Iteration  1500: 10778.05267244669
Iteration  1600: 10778.05267244669
Iteration  1700: 10778.05267244669
Iteration  1800: 10778.05267244669
Iteration  1900: 10778.05267244669
Iteration  2000: 10778.05267244669
Best for CMU-ES: [420.96874631 203.8142524  203.81425267 203.81425278 203.81425265
 203.81425303 203.81425237 203.81425258 203.81425267  65.54786516
 203.8142529  203.8142529  203.81425217 203.81425313 203.81425219
 203.81425245 203.81425272 203.8142526  203.81425259 203.81425286
 2

100%|██████████| 1000/1000 [12:24<00:00,  1.34it/s]

Best for ES: [ 203.78312024  203.80356957 -124.82813449 -302.55782076  420.90818884
  420.99976772  203.94062197   63.59552706 -302.48706636  420.93404817
  420.96999905 -302.71809207  420.71043637   65.44429863 -302.42031747
 -124.84842219  420.98319704  420.94086014  420.93425796 -302.3554006
 -302.57528181  204.03387721 -302.48547494 -302.52973882  421.0162026
 -302.55018641  420.90410191 -302.55845272   65.47024737  -25.91706953
 -302.01624628   65.52802285 -302.55846227  420.93910304   65.49209663
  420.87164415  420.93548085 -124.78283662  420.93872267  421.00599146
  203.82469097 -124.68561675 -499.9999121  -302.48423402 -302.5222161
  420.84876154 -124.7935521  -302.5889964   420.98688188 -302.53454819], value: 6871.854851427701
